In [34]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
#!/usr/bin/env python
# coding: utf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

import src.features.pabellon as pb

# Configuración de gráficos
sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

ANIO_INICIO = 2017
ANIO_TERMINO = 2035
COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

In [36]:
# Cargar datos
casos_area_de_influencia = pb.cargar_casos_area_de_influencia(
    "../data/interim/casos_teoricos_diagnosticos.xlsx"
)
porcentajes_de_quirurgicos = pb.cargar_porcentajes_de_quirurgicos(
    "../data/external/incidencias_y_prevalencias_INT.xlsx"
)

porcentajes_de_quirurgicos = porcentajes_de_quirurgicos.replace(
    ["Separado", "Preguntar"], "0"
).astype(float)

# Calcular casos quirúrgicos
DIAGS_AREA_DE_INFLUENCIA = casos_area_de_influencia.index
casos_quirurgicos = pb.calcular_casos_quirurgicos(
    casos_area_de_influencia, porcentajes_de_quirurgicos
)

Casos del área de influencia cargados y formateados:
+---+-------------+------------------------------------+-------------+----------------------+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|   | Diagnostico |      Diagnosticos Contenidos       | Estadística | Casos (Cada 100.000) |   Edad Inci

In [37]:
# Lee la base de GRD unida con Pabellon
grd_y_pabellon = pd.read_csv("../data/raw/5_duracion_int_q/grd_interno_y_pabellon.csv")
grd_y_pabellon["duracion"] = pd.to_timedelta(grd_y_pabellon["duracion"])

# Extrae los diagnosticos que se van a reasignar
diagnosticos_a_reasignar = casos_area_de_influencia.dropna(subset="Diagnosticos Contenidos")
diagnosticos_a_reasignar = diagnosticos_a_reasignar["Diagnosticos Contenidos"].str.split(", ")
diagnosticos_a_reasignar = diagnosticos_a_reasignar.reset_index()

# Reasigna diagnosticos y los agrupa
grd_y_pabellon = pb.reasignar_diagnosticos(
    grd_y_pabellon, "diag_01_principal_cod", diagnosticos_a_reasignar
)

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['I052', 'I080', 'I081'] a I051
Cambiando ['I352'] a I350
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


In [38]:
# Obtiene el resumen de duraciones de las int. q por diagnostico acumuladas
resumen_duraciones_int_q_acumuladas = (
    grd_y_pabellon.query("ano_de_egreso >= 2019 ")
    .groupby(["diag_01_principal_cod"])["duracion"]
    .describe()
)

# Obtiene el resumen de duraciones para los diagnosticos relevantes
resumen_duraciones_diags_relevantes = resumen_duraciones_int_q_acumuladas[
    resumen_duraciones_int_q_acumuladas.index.isin(DIAGS_AREA_DE_INFLUENCIA)
]["75%"]

In [46]:
# Calcular tiempo utilizado en pabellón en horas
tiempo_utilizado_pabellon_horas = pb.calcular_tiempo_utilizado_pabellon(
    casos_quirurgicos, resumen_duraciones_diags_relevantes
)

# Calcular horas laborales
horas_laborales = pb.calcular_horas_laborales(ANIO_INICIO, ANIO_TERMINO, 12)

# Calcular cantidad de pabellones necesarios
cantidad_de_pabellones_necesarios = pb.calcular_cantidad_de_pabellones_necesarios(
    tiempo_utilizado_pabellon_horas, horas_laborales
)

# Obtiene la suma total de pabellones
suma_total_pabellones = cantidad_de_pabellones_necesarios.sum()

Tiempo utilizado en pabellón calculado (en horas):
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      |        2017        |        2018        |        2019        |        2020        |        2021        |        2022        |        2023        |        2024        |        2025        |        2026        |        2027        |       2028        |        2029        |        2030        |        2031        |        2032        |        2033        |        2034        |        2035        |
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-

In [49]:
a_guardar = {
    "casos_area_de_influencia": casos_area_de_influencia,
    "porcentajes_quirurgicos": porcentajes_de_quirurgicos,
    "casos_quirurgicos": casos_quirurgicos,
    "resumen_duraciones_int_q_rel": resumen_duraciones_diags_relevantes,
    "tiempo_utilizado_pabellon": tiempo_utilizado_pabellon_horas,
    "horas_laborales_por_anio": horas_laborales,
    "cantidad_de_pabellones_desg": cantidad_de_pabellones_necesarios,
    "suma_total_pabellones": suma_total_pabellones,
}

In [57]:
resumen_area_de_estudio_minsal = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
).drop(columns="Porcentaje Hospitalizados de Ambulatorios")

In [58]:
resumen_area_de_estudio_minsal

,Diagnostico,Diagnosticos Contenidos,Estadística,Casos (Cada 100.000),Edad Incidencia,poblacion_ine_2035_area_de_estudio,casos_teoricos_ine_2035_area_de_estudio,porcentaje_fonasa_pais,casos_teoricos_fonasa_2035_area_de_estudio,Área de Influencia Formal,Área de Influencia Propuesta,casos_teoricos_fonasa_2035_area_de_influencia
0,"A152 - Tuberculosis del pulmón, confirmada his...",NaN,Incidencia,13.300,todos,21137769.0,2811.323277,0.804486,2261.669972,SSMO,SSMO,103.401089
1,C34N - Cáncer de Pulmón GES,"C341, C342, C343, C780, C782, D381",Incidencia,40.000,todos,21137769.0,8455.107600,0.804486,6802.014952,"SSMO, SSMC, Libertador General Bernardo O'Higg...","Metropolitana de Santiago, Libertador General ...",3148.137951
2,C381 - Tumor maligno del mediastino anterior,NaN,Incidencia,1.200,todos,21137769.0,253.653228,0.804486,204.060449,"SSMO, SSMC, Libertador General Bernardo O'Higg...","Metropolitana de Santiago, Libertador General ...",94.444139
3,E848 - Fibrosis quística con otras manifestaci...,NaN,Incidencia,11.100,recien_nacidos_vivos,212799.0,23.620689,0.804486,19.002512,Pais,Pais,19.002512
4,I051 - Insuficiencia mitral reumática,"I052, I080, I081",Incidencia,7.630,todos,21137769.0,1612.811775,0.804486,1297.484352,SSMO,SSMO,59.319572
5,I052 - Estenosis mitral con insuficiencia,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,0.804486,NaN,NaN,NaN,NaN
6,I080 - Trastornos de las válvulas mitral y aór...,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,0.804486,NaN,NaN,NaN,NaN
7,I081 - Trastornos de las válvulas mitral y tri...,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,0.804486,NaN,NaN,NaN,NaN
8,I232 - Defecto del tabique ventricular como co...,NaN,Incidencia,0.843,todos,21137769.0,178.191393,0.804486,143.352465,No existe,Metropolitana de Santiago,52.975517
9,I330 - Endocarditis infecciosa aguda y subaguda,NaN,Incidencia,2.500,todos,21137769.0,528.444225,0.804486,425.125935,No existe,SSMO,19.436295


In [51]:
["Diagnostico",
"Diagnosticos Contenidos",
"Estadística",
"Casos (Cada 100.000)",
"Edad Incidencia",
"poblacion_ine_2035_area_de_estudio",
"casos_teoricos_ine_2035_area_de_estudio",
"porcentaje_fonasa_pais",
"casos_teoricos_fonasa_2035_area_de_estudio",
"Área de Influencia Formal",
"Área de Influencia Propuesta",
"casos_teoricos_fonasa_2035_area_de_influencia",
"Porcentaje Hospitalizados de Ambulatorios",
"casos_hospitalizados_2035",
"egresos_por_paciente_agrupado_2017_a_2019",
"egresos_2035",
"dias_estada_promedio_agrupado_2017_a_2019",
"dias_estada_totales_2035",
"camas_totales_2035"]

,Diagnosticos Contenidos,Estadística,Casos (Cada 100.000),Edad Incidencia,Área de Influencia Formal,Área de Influencia Propuesta,Porcentaje Hospitalizados de Ambulatorios,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
Diagnostico,,,,,,,,,,,,,,,,,,,,,,,,,,
A152,NaN,Incidencia,13.300,todos,SSMO,SSMO,0.107856,87.360128,89.624170,92.124701,94.563776,95.891905,96.682000,97.332326,97.941854,98.529438,99.101191,99.655506,100.192511,100.707829,101.201977,101.679908,102.141686,102.581651,103.002052,103.401089
C34N,"C341, C342, C343, C780, C782, D381",Incidencia,40.000,todos,"SSMO, SSMC, Libertador General Bernardo O'Higg...","Metropolitana de Santiago, Libertador General ...",0.353535,2701.860474,2763.141262,2829.693102,2895.442067,2933.634072,2957.229235,2977.249600,2996.028526,3013.989962,3031.185266,3047.585064,3063.145540,3077.817615,3091.566764,3104.579007,3116.775211,3128.096003,3138.546468,3148.137951
C381,NaN,Incidencia,1.200,todos,"SSMO, SSMC, Libertador General Bernardo O'Higg...","Metropolitana de Santiago, Libertador General ...",0.353535,81.055814,82.894238,84.890793,86.863262,88.009022,88.716877,89.317488,89.880856,90.419699,90.935558,91.427552,91.894366,92.334528,92.747003,93.137370,93.503256,93.842880,94.156394,94.444139
E848,NaN,Incidencia,11.100,recien_nacidos_vivos,Pais,Pais,0.326246,20.755073,20.329568,20.532007,20.960726,21.206920,21.236746,21.164950,21.048863,20.901075,20.726587,20.529953,20.315995,20.089178,19.852985,19.765741,19.572232,19.377384,19.185751,19.002512
I051,"I052, I080, I081",Incidencia,7.630,todos,SSMO,SSMO,0.403037,50.117126,51.415971,52.850486,54.249745,55.011672,55.464937,55.838019,56.187695,56.524783,56.852789,57.170791,57.478862,57.774492,58.057976,58.332157,58.597073,58.849473,59.090651,59.319572
I052,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I080,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I081,NaN,Contenido en Mitral (I051),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
I232,NaN,Incidencia,0.843,todos,No existe,Metropolitana de Santiago,0.403037,44.975781,46.139012,47.410701,48.667804,49.370933,49.781386,50.121644,50.438740,50.741316,51.030523,51.305911,51.566667,51.811908,52.041223,52.257438,52.459565,52.646519,52.818469,52.975517


In [50]:
with pd.ExcelWriter("../data/interim/estimacion_pabellones_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)